In [3]:
# Load in the data
import pandas as pd

file = "pendulum_data.csv"

df = pd.read_csv(file)
print("Loaded:", file)
print("Shape:", df.shape)
df.head()

Loaded: pendulum_data.csv
Shape: (99499, 26)


,company,document_id,post_id,post_title,post_url,media_url,upload_date,impression_count,pred_impressions,impression_count_comb,...,comment_count,share_count,communities,snippet_lang,snippet_text,snippet_source,duration,offset,sentiment,sentiment_score
0,Housing Market (Buying/Selling Homes),bcdda782-5984-36d7-b747-adbe88b56022,5a16d0c50fb23295adb072c861be5e33,NaN,https://www.tiktok.com/@thisistechtoday/video/...,https://www.tiktok.com/@thisistechtoday/video/...,2025-04-03T20:49:25,21800000,1.785211e+04,21800000.0,...,82.0,20.0,NaN,en,"type, I decided to make some videos at the pla...",description,NaN,NaN,Positive,0.941140
1,Housing Market (Buying/Selling Homes),ef28b6bf-33d3-3e34-9ca8-5d1bb495b020,f727cbb3f43e389a93a025a2aa1337a3,Zillow | Are We Doing This?,https://www.youtube.com/watch?v=mAxS5WZqcho&t=0s,https://www.youtube.com/watch?v=mAxS5WZqcho,2025-02-28T23:01:34,18626940,1.559422e+04,18626940.0,...,NaN,NaN,NaN,en,( music playing ) I'm thinking of <strong>buyi...,transcript,14.0,0.0,Neutral,0.587764
2,Housing Market (Buying/Selling Homes),407ddf77-0117-361b-a255-881b1bb45caa,a012e9ad16713e2ebe48361fb4c98b3f,NaN,https://www.tiktok.com/@lic.manene/video/75459...,https://www.tiktok.com/@lic.manene/video/75459...,2025-09-03T20:01:25,13900000,4.545480e+04,13900000.0,...,9041.0,736100.0,NaN,en,legal advice on the consequences of <strong>se...,image_caption,NaN,NaN,Neutral,0.637891
3,Housing Market (Buying/Selling Homes),86e453de-cc84-3fc4-9077-232f8e8b7110,ff2247ead37e351cae0c9987802ae8d3,We’re Moving to Hawaii *emotional*,https://www.youtube.com/watch?v=49OSWOITnNA&t=...,https://www.youtube.com/watch?v=49OSWOITnNA,2025-08-31T16:58:39,10080500,4.488331e+06,10080500.0,...,NaN,NaN,"[77, 77]",en,special gift. All right. Special gift. You're ...,transcript,16.0,242.0,Positive,0.544222
4,Housing Market (Buying/Selling Homes),1d3ff552-d324-391a-9537-7d5b9b8cd16b,f23749a3b03a376591a200b7e4c00c22,NaN,https://twitter.com/jacksonhinklle/status/1884...,https://twitter.com/jacksonhinklle/status/1884...,2025-01-29T19:33:05,8062540,5.134603e+05,513460.0,...,4968.0,13798.0,[],en,"in infrastructure, high speed rail, healthcare...",description,NaN,NaN,Neutral,0.773371


In [5]:
# Keep only the necessary columns
desired_cols = ['company', 'upload_date', 'impression_count_comb', 'platform', 'snippet_text']

present = [c for c in desired_cols if c in df.columns]

df = df[present].copy()

# Rename company to topic
df.rename(columns={"company": "topic"}, inplace=True)

print("Kept columns:", df.columns.tolist())

df.head()

Kept columns: ['topic', 'upload_date', 'impression_count_comb', 'platform', 'snippet_text']


,topic,upload_date,impression_count_comb,platform,snippet_text
0,Housing Market (Buying/Selling Homes),2025-04-03T20:49:25,21800000.0,TikTok,"type, I decided to make some videos at the pla..."
1,Housing Market (Buying/Selling Homes),2025-02-28T23:01:34,18626940.0,YouTube,( music playing ) I'm thinking of <strong>buyi...
2,Housing Market (Buying/Selling Homes),2025-09-03T20:01:25,13900000.0,TikTok,legal advice on the consequences of <strong>se...
3,Housing Market (Buying/Selling Homes),2025-08-31T16:58:39,10080500.0,YouTube,special gift. All right. Special gift. You're ...
4,Housing Market (Buying/Selling Homes),2025-01-29T19:33:05,513460.0,Twitter,"in infrastructure, high speed rail, healthcare..."


In [7]:
# Load score dates
schedule_file = "interview_schedule_2025.csv"
schedule_df = pd.read_csv(schedule_file)
print("Loaded:", schedule_file)
print("Shape:", schedule_df.shape)
schedule_df.head()

Loaded: interview_schedule_2025.csv
Shape: (132, 4)


,Month,Year,Start Date,End Date
0,1,2015,2-Jan-15,26-Jan-15
1,2,2015,29-Jan-15,23-Feb-15
2,3,2015,26-Feb-15,23-Mar-15
3,4,2015,26-Mar-15,25-Apr-15
4,5,2015,29-Apr-15,23-May-15


In [9]:
# Convert date columns to datetimes (explicit formats: upload_date ISO, schedule day-month-year like 17-Dec-24)
df['upload_date'] = pd.to_datetime(df['upload_date'], format="%Y-%m-%dT%H:%M:%S", errors='coerce')
schedule_df['Start Date'] = pd.to_datetime(schedule_df['Start Date'], format="%d-%b-%y", errors='coerce')
schedule_df['End Date'] = pd.to_datetime(schedule_df['End Date'], format="%d-%b-%y", errors='coerce')

# Prepare Month/Year columns (nullable integers)
df['Month'] = pd.NA
df['Year'] = pd.NA

def _safe_month(val, fallback_dt):
    if pd.isna(val):
        return (pd.NA if pd.isna(fallback_dt) else int(fallback_dt.month))
    try:
        return int(val)
    except Exception:
        # try parsing textual month (e.g. "Dec" / "December")
        try:
            parsed = pd.to_datetime(str(val), errors='coerce')
            if pd.notna(parsed):
                return int(parsed.month)
        except Exception:
            pass
    return (pd.NA if pd.isna(fallback_dt) else int(fallback_dt.month))

def _safe_year(val, fallback_dt):
    if pd.isna(val):
        return (pd.NA if pd.isna(fallback_dt) else int(fallback_dt.year))
    try:
        return int(val)
    except Exception:
        try:
            parsed = pd.to_datetime(str(val), errors='coerce')
            if pd.notna(parsed):
                return int(parsed.year)
        except Exception:
            pass
    return (pd.NA if pd.isna(fallback_dt) else int(fallback_dt.year))

# For each schedule interval, assign Month and Year to matching upload_date rows
for _, srow in schedule_df[['Start Date', 'End Date', 'Month', 'Year']].dropna(subset=['Start Date', 'End Date']).iterrows():
    start, end = srow['Start Date'], srow['End Date']
    mon_raw, yr_raw = srow.get('Month'), srow.get('Year')
    mon = _safe_month(mon_raw, start)
    yr = _safe_year(yr_raw, start)
    mask = (df['upload_date'] >= start) & (df['upload_date'] <= end)
    df.loc[mask, 'Month'] = mon
    df.loc[mask, 'Year'] = yr

# convert to nullable integer dtypes for convenience
df['Month'] = df['Month'].astype('Int64')
df['Year'] = df['Year'].astype('Int64')

# Show result
print("Assigned months:", sorted(df['Month'].dropna().unique().tolist()))
df.head()

Assigned months: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


,topic,upload_date,impression_count_comb,platform,snippet_text,Month,Year
0,Housing Market (Buying/Selling Homes),2025-04-03 20:49:25,21800000.0,TikTok,"type, I decided to make some videos at the pla...",4,2025
1,Housing Market (Buying/Selling Homes),2025-02-28 23:01:34,18626940.0,YouTube,( music playing ) I'm thinking of <strong>buyi...,3,2025
2,Housing Market (Buying/Selling Homes),2025-09-03 20:01:25,13900000.0,TikTok,legal advice on the consequences of <strong>se...,9,2025
3,Housing Market (Buying/Selling Homes),2025-08-31 16:58:39,10080500.0,YouTube,special gift. All right. Special gift. You're ...,9,2025
4,Housing Market (Buying/Selling Homes),2025-01-29 19:33:05,513460.0,Twitter,"in infrastructure, high speed rail, healthcare...",2,2025


In [11]:
# Remove rows with upload_date on or before 2024-12-16 (preserve rows with missing upload_date)
threshold = pd.to_datetime("2024-12-16")

mask_remove = df['upload_date'].notna() & (df['upload_date'] <= threshold)
removed_count = int(mask_remove.sum())

df = df.loc[~mask_remove].copy()

print(f"Removed {removed_count} rows with upload_date on or before {threshold.date()}")
print("New shape:", df.shape)
df.head()

Removed 18042 rows with upload_date on or before 2024-12-16
New shape: (81457, 7)


,topic,upload_date,impression_count_comb,platform,snippet_text,Month,Year
0,Housing Market (Buying/Selling Homes),2025-04-03 20:49:25,21800000.0,TikTok,"type, I decided to make some videos at the pla...",4,2025
1,Housing Market (Buying/Selling Homes),2025-02-28 23:01:34,18626940.0,YouTube,( music playing ) I'm thinking of <strong>buyi...,3,2025
2,Housing Market (Buying/Selling Homes),2025-09-03 20:01:25,13900000.0,TikTok,legal advice on the consequences of <strong>se...,9,2025
3,Housing Market (Buying/Selling Homes),2025-08-31 16:58:39,10080500.0,YouTube,special gift. All right. Special gift. You're ...,9,2025
4,Housing Market (Buying/Selling Homes),2025-01-29 19:33:05,513460.0,Twitter,"in infrastructure, high speed rail, healthcare...",2,2025


In [13]:
# Count missing values per column (count and percent)
missing_counts = df.isna().sum()
missing_pct = (df.isna().mean() * 100).round(2)

missing_summary = pd.DataFrame({
    "missing_count": missing_counts,
    "missing_pct": missing_pct
}).sort_values("missing_count", ascending=False)

print(missing_summary)

                       missing_count  missing_pct
Month                           2231         2.74
Year                            2231         2.74
topic                              0         0.00
upload_date                        0         0.00
impression_count_comb              0         0.00
platform                           0         0.00
snippet_text                       0         0.00


In [15]:
# Drop any rows that contain missing values
before_count = len(df)
df = df.dropna().copy()
removed_count = before_count - len(df)

print(f"Removed {removed_count} rows with any missing values. New shape: {df.shape}")
df.head()

missing_counts = df.isna().sum()
missing_pct = (df.isna().mean() * 100).round(2)
missing_summary = pd.DataFrame({
    "missing_count": missing_counts,
    "missing_pct": missing_pct
}).sort_values("missing_count", ascending=False)
print(missing_summary)

Removed 2231 rows with any missing values. New shape: (79226, 7)
                       missing_count  missing_pct
topic                              0          0.0
upload_date                        0          0.0
impression_count_comb              0          0.0
platform                           0          0.0
snippet_text                       0          0.0
Month                              0          0.0
Year                               0          0.0


In [21]:
from transformers import pipeline, AutoTokenizer
import torch

HF_MODEL_PRIMARY   = "cardiffnlp/twitter-roberta-base-sentiment-latest"

model_name = HF_MODEL_PRIMARY if 'HF_MODEL_PRIMARY' in globals() else "cardiffnlp/twitter-roberta-base-sentiment-latest"
text_col = 'snippet_text'
if text_col not in df.columns:
    raise KeyError(f"Expected text column '{text_col}' not found in df")

device = 0 if torch.cuda.is_available() else -1

# load tokenizer with a safe max length and build pipeline with it
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
# enforce a reasonable max length (Roberta-based models typically 512)
tokenizer.model_max_length = min(getattr(tokenizer, "model_max_length", 512), 512)

sent_pipe = pipeline(
    "sentiment-analysis",
    model=model_name,
    tokenizer=tokenizer,
    device=device,
    return_all_scores=True
)

batch_size = 32
scores = []
n = len(df)

for i in range(0, n, batch_size):
    batch_texts = df[text_col].iloc[i:i+batch_size].fillna("").astype(str).tolist()
    # ensure inputs are truncated/padded to tokenizer.model_max_length to avoid oversized tensors
    results = sent_pipe(batch_texts, truncation=True, padding=True, max_length=tokenizer.model_max_length)
    for res in results:
        label_scores = {entry['label'].lower(): entry['score'] for entry in res}
        pos = label_scores.get('positive', 0.0)
        neg = label_scores.get('negative', 0.0)
        scores.append(pos - neg)

if len(scores) != n:
    raise RuntimeError(f"Score count ({len(scores)}) does not match df length ({n})")

df['sentiment'] = scores
print("Added 'sentiment' column to df (positive-negative score).")
df.head()
#

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
/opt/anaconda3/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `retur

Added 'sentiment' column to df (positive-negative score).


,topic,upload_date,impression_count_comb,platform,snippet_text,Month,Year,sentiment,sentiment_weighted
0,Housing Market (Buying/Selling Homes),2025-04-03 20:49:25,21800000.0,TikTok,"type, I decided to make some videos at the pla...",4,2025,0.933541,15.774429
1,Housing Market (Buying/Selling Homes),2025-02-28 23:01:34,18626940.0,YouTube,( music playing ) I'm thinking of <strong>buyi...,3,2025,0.154169,2.580812
2,Housing Market (Buying/Selling Homes),2025-09-03 20:01:25,13900000.0,TikTok,legal advice on the consequences of <strong>se...,9,2025,-0.194897,-3.205542
3,Housing Market (Buying/Selling Homes),2025-08-31 16:58:39,10080500.0,YouTube,special gift. All right. Special gift. You're ...,9,2025,0.599563,9.668617
4,Housing Market (Buying/Selling Homes),2025-01-29 19:33:05,513460.0,Twitter,"in infrastructure, high speed rail, healthcare...",2,2025,0.122555,1.611473


In [45]:
import numpy as np

# Compute log(1 + count) and weight sentiment
df['sentiment_weighted'] = df['sentiment'] * np.log1p(df['impression_count_comb'])

# quick check
print(df[['sentiment', 'impression_count_comb', 'sentiment_weighted']].head())

   sentiment  impression_count_comb  sentiment_weighted
0   0.933541             21800000.0           15.774429
1   0.154169             18626940.0            2.580812
2  -0.194897             13900000.0           -3.205542
3   0.599563             10080500.0            9.668617
4   0.122555               513460.0            1.611473


In [47]:
# Aggregate average sentiment_weighted for rows that have Month, Year, and topic
agg_df = (
    df.dropna(subset=['Month', 'Year', 'topic'])
      .groupby(['Year', 'Month', 'topic'], as_index=False)['sentiment_weighted']
      .mean()
      .rename(columns={'sentiment_weighted': 'sentiment_weighted_avg'})
)

# optional: sort for readability
agg_df = agg_df.sort_values(['Year', 'Month', 'topic']).reset_index(drop=True)

print("Aggregated shape:", agg_df.shape)
agg_df.head()

Aggregated shape: (110, 4)


,Year,Month,topic,sentiment_weighted_avg
0,2025,1,Business and Economic Conditions,-2.424275
1,2025,1,Durable Goods and Big Purchases,-0.985370
2,2025,1,Gasoline and Energy Prices,-3.708268
3,2025,1,Government Policy and Inflation Control,-2.572633
4,2025,1,Housing Market (Buying/Selling Homes),-2.474178


In [49]:
# Pivot the aggregated data so each row is a Year-Month and each column is a topic with the weighted score
if 'agg_df' not in globals():
    raise NameError("Expected 'agg_df' to exist from previous cell (aggregated Year, Month, topic).")

wide_df = (
    agg_df
    .pivot_table(index=['Year', 'Month'], columns='topic', values='sentiment_weighted_avg', aggfunc='first')
    .sort_index()
)

# Convert Year/Month MultiIndex to a PeriodIndex (monthly) for easier time-based handling
try:
    wide_df.index = pd.PeriodIndex(
        year=wide_df.index.get_level_values('Year').astype(int),
        month=wide_df.index.get_level_values('Month').astype(int),
        freq='M'
    )
    wide_df.index.name = 'Period'
except Exception:
    # If conversion fails, keep the Year/Month MultiIndex (ensuring integer dtypes)
    wide_df = wide_df.rename_axis(index=['Year', 'Month'])

print("Wide dataframe shape:", wide_df.shape)
wide_df.head()

Wide dataframe shape: (11, 10)


/var/folders/ng/tvbgv2jn62n0krpw4r35yyf40000gn/T/ipykernel_80285/3055281622.py:13: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  wide_df.index = pd.PeriodIndex(


topic       Business and Economic Conditions  Durable Goods and Big Purchases  \
Year Month                                                                      
2025 1                             -2.424275                        -0.985370   
     2                             -1.989103                        -0.514461   
     3                             -3.108605                        -3.296978   
     4                             -3.772806                        -3.358477   
     5                             -2.124024                        -3.034095   

topic       Gasoline and Energy Prices  \
Year Month                               
2025 1                       -3.708268   
     2                       -3.672223   
     3                       -4.129982   
     4                       -4.195443   
     5                       -3.667764   

topic       Government Policy and Inflation Control  \
Year Month                                            
2025 1                                    -2.572633   
     2                                    -2.181203   
     3                                    -2.598650   
     4                                    -3.735365   
     5                                    -3.601755   

topic       Housing Market (Buying/Selling Homes)  Income Expectations  \
Year Month                                                               
2025 1                                  -2.474178             0.995655   
     2                                  -1.907912             1.273276   
     3                                  -2.650578             0.953472   
     4                                  -2.927299             0.792706   
     5                                  -2.731952            -0.016559   

topic       Inflation and Prices  Investments and Stock Market Confidence  \
Year Month                                                                  
2025 1                 -3.962340                                -0.059577   
     2                 -3.662704                                -0.341005   
     3                 -4.070910                                -1.871086   
     4                 -4.083152                                -2.796518   
     5                 -3.549505                                -1.225569   

topic       Personal Financial Situation  Unemployment and Job Security  
Year Month                                                               
2025 1                         -1.741165                      -3.915001  
     2                         -2.439821                      -3.692086  
     3                         -2.363554                      -4.246966  
     4                         -2.451119                      -4.067519  
     5                         -2.111926                      -3.362209

In [51]:
# Load MCSI scores
scores_file = "scores.csv"
scores_df = pd.read_csv(scores_file)
print("Loaded:", scores_file)
print("Shape:", scores_df.shape)
scores_df.head()

Loaded: scores.csv
Shape: (131, 3)


,Month,Year,score
0,1,2015,98.1
1,2,2015,95.4
2,3,2015,93.0
3,4,2015,95.9
4,5,2015,90.7


In [53]:
# Add MCSI scores to the main data frame
wide_df = wide_df.merge(
    scores_df,
    on=['Month', 'Year'],
    how='left'
)

wide_df
# Now we have one row for each month. That row has 10 features (an aggregated and weighted score for each topic) and an outcome variable (the MCSI score for that month)

,Month,Year,Business and Economic Conditions,Durable Goods and Big Purchases,Gasoline and Energy Prices,Government Policy and Inflation Control,Housing Market (Buying/Selling Homes),Income Expectations,Inflation and Prices,Investments and Stock Market Confidence,Personal Financial Situation,Unemployment and Job Security,score
0,1,2025,-2.424275,-0.985370,-3.708268,-2.572633,-2.474178,0.995655,-3.962340,-0.059577,-1.741165,-3.915001,71.7
1,2,2025,-1.989103,-0.514461,-3.672223,-2.181203,-1.907912,1.273276,-3.662704,-0.341005,-2.439821,-3.692086,64.7
2,3,2025,-3.108605,-3.296978,-4.129982,-2.598650,-2.650578,0.953472,-4.070910,-1.871086,-2.363554,-4.246966,57.0
3,4,2025,-3.772806,-3.358477,-4.195443,-3.735365,-2.927299,0.792706,-4.083152,-2.796518,-2.451119,-4.067519,52.2
4,5,2025,-2.124024,-3.034095,-3.667764,-3.601755,-2.731952,-0.016559,-3.549505,-1.225569,-2.111926,-3.362209,52.2
5,6,2025,-1.355387,-1.195504,-4.208126,-2.996546,-2.453519,0.943184,-2.633266,-0.070558,-2.646174,-5.163141,60.7
6,7,2025,-2.493379,-1.980791,-3.281829,-2.881100,-2.473568,-0.130992,-3.838626,0.955328,-2.228622,-4.276139,61.7
7,8,2025,-1.814426,-1.131614,-3.741111,-2.717026,-2.745103,0.097175,-4.036495,0.328349,-2.335906,-4.736820,58.2
8,9,2025,-2.678492,-0.802823,-3.502596,-3.204801,-2.316663,0.373122,-3.472888,0.034123,-3.043270,-4.984763,55.1
9,10,2025,-2.105846,-2.211884,-3.588996,-2.734363,-3.057141,0.078202,-3.497183,0.397582,-2.505635,-4.258151,53.6


In [55]:
# Create lag feature (okay because we are splitting temporally for train/validate sets)
wide_df['prev_score'] = wide_df['score'].shift(1)

# Manually add back the score from 12/2024 to minimize data loss
wide_df.loc[(wide_df['Year'] == 2025) & (wide_df['Month'] == 1), 'prev_score'] = 74

wide_df

,Month,Year,Business and Economic Conditions,Durable Goods and Big Purchases,Gasoline and Energy Prices,Government Policy and Inflation Control,Housing Market (Buying/Selling Homes),Income Expectations,Inflation and Prices,Investments and Stock Market Confidence,Personal Financial Situation,Unemployment and Job Security,score,prev_score
0,1,2025,-2.424275,-0.985370,-3.708268,-2.572633,-2.474178,0.995655,-3.962340,-0.059577,-1.741165,-3.915001,71.7,74.0
1,2,2025,-1.989103,-0.514461,-3.672223,-2.181203,-1.907912,1.273276,-3.662704,-0.341005,-2.439821,-3.692086,64.7,71.7
2,3,2025,-3.108605,-3.296978,-4.129982,-2.598650,-2.650578,0.953472,-4.070910,-1.871086,-2.363554,-4.246966,57.0,64.7
3,4,2025,-3.772806,-3.358477,-4.195443,-3.735365,-2.927299,0.792706,-4.083152,-2.796518,-2.451119,-4.067519,52.2,57.0
4,5,2025,-2.124024,-3.034095,-3.667764,-3.601755,-2.731952,-0.016559,-3.549505,-1.225569,-2.111926,-3.362209,52.2,52.2
5,6,2025,-1.355387,-1.195504,-4.208126,-2.996546,-2.453519,0.943184,-2.633266,-0.070558,-2.646174,-5.163141,60.7,52.2
6,7,2025,-2.493379,-1.980791,-3.281829,-2.881100,-2.473568,-0.130992,-3.838626,0.955328,-2.228622,-4.276139,61.7,60.7
7,8,2025,-1.814426,-1.131614,-3.741111,-2.717026,-2.745103,0.097175,-4.036495,0.328349,-2.335906,-4.736820,58.2,61.7
8,9,2025,-2.678492,-0.802823,-3.502596,-3.204801,-2.316663,0.373122,-3.472888,0.034123,-3.043270,-4.984763,55.1,58.2
9,10,2025,-2.105846,-2.211884,-3.588996,-2.734363,-3.057141,0.078202,-3.497183,0.397582,-2.505635,-4.258151,53.6,55.1


In [57]:
# Aggregate to internal and external sentiment
internal_columns = ['Durable Goods and Big Purchases', 'Gasoline and Energy Prices', 'Income Expectations',
                    'Inflation and Prices','Personal Financial Situation','Unemployment and Job Security']
external_columns = ['Business and Economic Conditions', 'Government Policy and Inflation Control',
                    'Housing Market (Buying/Selling Homes)','Investments and Stock Market Confidence']
wide_df['internal_sentiment'] = wide_df[internal_columns].mean(axis=1)
wide_df['external_sentiment'] = wide_df[external_columns].mean(axis=1)

wide_df = wide_df.drop(['Durable Goods and Big Purchases', 'Gasoline and Energy Prices', 'Income Expectations',
                    'Inflation and Prices','Personal Financial Situation','Unemployment and Job Security',
                    'Business and Economic Conditions', 'Government Policy and Inflation Control',
                    'Housing Market (Buying/Selling Homes)','Investments and Stock Market Confidence'], axis=1)

wide_df

,Month,Year,score,prev_score,internal_sentiment,external_sentiment
0,1,2025,71.7,74.0,-2.219415,-1.882666
1,2,2025,64.7,71.7,-2.118003,-1.604806
2,3,2025,57.0,64.7,-2.859153,-2.557229
3,4,2025,52.2,57.0,-2.893834,-3.307997
4,5,2025,52.2,52.2,-2.623676,-2.420825
5,6,2025,60.7,52.2,-2.483838,-1.719003
6,7,2025,61.7,60.7,-2.622833,-1.723180
7,8,2025,58.2,61.7,-2.647462,-1.737052
8,9,2025,55.1,58.2,-2.572203,-2.041458
9,10,2025,53.6,55.1,-2.663941,-1.874942


In [59]:
# Create training and validation splits to fit models
train_df = wide_df[wide_df['Month'] < 9]
validate_df = wide_df[(wide_df['Month'] >= 9) & (wide_df['Month'] < 11)]

In [61]:
# Splitting features and outcomes
# --- Prepare data ---
# Sort for readability
train_df = train_df.sort_values(by=["Year", "Month"]).reset_index(drop=True)
validate_df = validate_df.sort_values(by=["Year", "Month"]).reset_index(drop=True)

# Define features and target
exclude_cols = ["Month", "Year", "score"]
X_train = train_df.drop(columns=exclude_cols)
y_train = train_df["score"]

X_val = validate_df.drop(columns=exclude_cols)
y_val = validate_df["score"]

In [91]:
# Ridge Model
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error

# --- Train Ridge model ---
ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)

# --- Predictions ---
train_df["predicted_score"] = ridge.predict(X_train)
validate_df["predicted_score"] = ridge.predict(X_val)

# --- Summary ---
print("=== Ridge Regression Summary ===")
print(f"Alpha (regularization strength): {ridge.alpha}")
print(f"R² (train): {ridge.score(X_train, y_train):.4f}")
print(f"R² (validation): {r2_score(y_val, validate_df['predicted_score']):.4f}")
import numpy as np
rmse_val = np.sqrt(mean_squared_error(y_val, validate_df['predicted_score']))
print(f"RMSE (validation): {rmse_val:.4f}")
print("\n--- Coefficients ---")
for name, coef in zip(X_train.columns, ridge.coef_):
    print(f"{name:20s} : {coef: .4f}")
print(f"\nIntercept: {ridge.intercept_:.4f}\n")

# --- Output Tables ---
print("=== Train Predictions (first 10) ===")
print(train_df[["Year", "Month", "score", "predicted_score"]].head(10))

print("\n=== Validation Predictions (first 10) ===")
print(validate_df[["Year", "Month", "score", "predicted_score"]].head(10))

=== Ridge Regression Summary ===
Alpha (regularization strength): 1.0
R² (train): 0.7943
R² (validation): -18.5827
RMSE (validation): 3.3189

--- Coefficients ---
prev_score           :  0.4974
internal_sentiment   :  1.8746
external_sentiment   :  3.2678

Intercept: 40.7923

=== Train Predictions (first 10) ===
   Year  Month  score  predicted_score
0  2025      1   71.7        67.289368
1  2025      2   64.7        67.243371
2  2025      3   57.0        59.259701
3  2025      4   52.2        52.911152
4  2025      5   52.2        53.929001
5  2025      6   60.7        56.484535
6  2025      7   61.7        60.438471
7  2025      8   58.2        60.844400

=== Validation Predictions (first 10) ===
   Year  Month  score  predicted_score
0  2025      9   55.1        58.249750
1  2025     10   53.6        57.079883


In [93]:
## Manually writing formula because API data comes in a very different format
def ridge(df):
    coef_prev = 0.4974
    coef_internal = 1.8746
    coef_external = 3.2678
    intercept = 40.7923
    ridge_score = (
        coef_prev * df["prev_score"]
        + coef_internal * df["internal_sentiment"]
        + coef_external * df["external_sentiment"]
        + intercept
    )
    return ridge_score

In [95]:
# --- Train Lasso model ---
lasso = Lasso(alpha=1.0)
lasso.fit(X_train, y_train)

# --- Predictions ---
train_df["predicted_score"] = lasso.predict(X_train)
validate_df["predicted_score"] = lasso.predict(X_val)

# --- Summary ---
print("=== Lasso Regression Summary ===")
print(f"Alpha (regularization strength): {lasso.alpha}")
print(f"R² (train): {lasso.score(X_train, y_train):.4f}")
print(f"R² (validation): {r2_score(y_val, validate_df['predicted_score']):.4f}")

rmse_val = np.sqrt(mean_squared_error(y_val, validate_df['predicted_score']))
print(f"RMSE (validation): {rmse_val:.4f}")
print("\n--- Coefficients ---")
for name, coef in zip(X_train.columns, ridge.coef_):
    print(f"{name:20s} : {coef: .4f}")
print(f"\nIntercept: {lasso.intercept_:.4f}\n")

# --- Output Tables ---
print("=== Train Predictions (first 10) ===")
print(train_df[["Year", "Month", "score", "predicted_score"]].head(10))

print("\n=== Validation Predictions (first 10) ===")
print(validate_df[["Year", "Month", "score", "predicted_score"]].head(10))

=== Lasso Regression Summary ===
Alpha (regularization strength): 1.0
R² (train): 0.7026
R² (validation): -13.7877
RMSE (validation): 2.8841

--- Coefficients ---


AttributeError: 'function' object has no attribute 'coef_'

In [97]:
import statsmodels.api as sm

# --- Prepare design matrices (add intercept) ---
X_train_ols = sm.add_constant(X_train, has_constant='add')
X_val_ols   = sm.add_constant(X_val,   has_constant='add')

# --- Fit OLS ---
ols_model = sm.OLS(y_train, X_train_ols).fit()

# --- Predictions ---
train_df["predicted_score"]    = ols_model.predict(X_train_ols)
validate_df["predicted_score"] = ols_model.predict(X_val_ols)

# --- Summary ---
print("=== OLS Regression Summary ===")
print(ols_model.summary())  # full table

# Also print quick metrics to mirror your Lasso block
print(f"\nR² (train): {ols_model.rsquared:.4f}")
r2_val   = r2_score(y_val, validate_df["predicted_score"])
rmse_val = np.sqrt(mean_squared_error(y_val, validate_df["predicted_score"]))
print(f"R² (validation): {r2_val:.4f}")
print(f"RMSE (validation): {rmse_val:.4f}")

# --- Coefficients ---
print("\n--- Coefficients ---")
# params is a Series indexed by ['const', <feature names>]
for name, coef in ols_model.params.items():
    print(f"{name:20s} : {coef: .4f}")

print(f"\nIntercept: {ols_model.params.get('const', np.nan):.4f}\n")

# --- Output Tables ---
print("=== Train Predictions (first 10) ===")
print(train_df[["Year", "Month", "score", "predicted_score"]].head(10))

print("\n=== Validation Predictions (first 10) ===")
print(validate_df[["Year", "Month", "score", "predicted_score"]].head(10))

=== OLS Regression Summary ===
                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                       0.828
Model:                            OLS   Adj. R-squared:                  0.699
Method:                 Least Squares   F-statistic:                     6.412
Date:                Sun, 16 Nov 2025   Prob (F-statistic):             0.0523
Time:                        19:59:58   Log-Likelihood:                -18.757
No. Observations:                   8   AIC:                             45.51
Df Residuals:                       4   BIC:                             45.83
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const

In [99]:
import joblib
joblib.dump(ols_model, "ols.joblib")

['ols.joblib']

In [101]:
joblib.dump(lasso, "lasso.joblib")

['lasso.joblib']

In [103]:
joblib.dump(ridge, "ridge.joblib")

['ridge.joblib']

In [105]:
## After I have more data, try without the lag component, then with it

## Once we get more data, we might be able to expand to a variety of other models

## Then we pick a model and save the trained model to github and deploy it using a streamlit app. We can put all of the preprocessing 
## stuff into a function so that new data is cleaned as is appropriate and turned into a score. Explain exactly what data is going in with 
## The API stuff I did.